In [1]:
import numpy as np
import pandas as pd 
import sys, os, psutil, io
import gzip
import time 
import warnings

from pybedtools import BedTool
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')

In [2]:
synteny_condensed=pd.read_csv('/pollard/home/shzhang/data/synteny/hg38_mm10_condensed.txt', sep='\t')

In [3]:
synteny_condensed

,hg_chr,hg_start,hg_end,hg_len,hg_seq,mm_chr,mm_start,mm_end,mm_len,mm_seq,total_align_len
0,chr1,56554,58923,2370,CACATTATTCTGCAGTTTTTATCTTGCTTACGTATTTTTGGAATGT...,chr2,111473169,111475499,2331,cacattgtattgta--ctctattatgttt-tattgtattatattat...,2518
1,chr1,60916,61124,209,TTAATATTTCCCACTTCATCATTACTAACCTCACTCACTCTGAACT...,chr2,111476095,111476298,204,ttAATAT--CCCAGTGCACC-TCACTACTCTGGCTCACGCTGAGCC...,209
2,chr1,61125,61835,711,AGTTCAAGACGTTTAATCAAGACCCTTCACTTACTTAACTATACCT...,chr2,111477065,111477685,621,aGCCCAAGAAAATTAAACAAATCCCTTTACTAATTTAAATGTATCT...,731
3,chr1,61943,61971,29,ctccgcctcccggggtcaagctattctcc,chr2,111477686,111477713,28,CTCTGCTTTGTGTGGAGAAG-TATTGTCC,29
4,chr1,62160,62467,308,tgtaaaactttttcctaatttaacagaaaaataatagtattatatt...,chr2,111477762,111478104,343,tgtaa----tttttcttatttaaCA-AGAAATGAGA-------ATT...,372
...,...,...,...,...,...,...,...,...,...,...,...
1229834,chrY,57107075,57107468,394,TGATTATGTTCGTTGTGACTGACAGCTGGATTGtagtggttaagaa...,chrX_GL456233_random,316168,316535,368,TAATGATGTTGATTGTGACTAGCAACAAGACTGtagtgattaagag...,397
1229835,chrY,57125268,57126152,885,aaagctgcagtgagtcgtaattgtgccactgcactccagcttgggc...,chrX_GL456233_random,321571,322437,867,aaggctgtatcg---cctaatagtgccacttcccatgagccaagca...,954
1229836,chrY,57126153,57126418,266,AGGTTGTTACTGCAAATCATTCATGTAGTCAGTGCCAGGGGTTAAA...,chrX_GL456233_random,322699,322963,265,ATATTGTTACTACAAATCATTCATATAGTCAGTGTGAAGAATAAAA...,266
1229837,chrY,57126419,57126561,143,CTTTTAATAGTGAAACACATGACCAGGCAGTGGC-TTTCCTATGAA...,chrX_GL456233_random,323295,323435,141,CTTTTCATCATAAAGTACATGACCAGGAAATATCTTTTCTTGTGAA...,150


In [4]:
# #check that lengths of seqs and alignments are correct  
# synteny_condensed['hg_dash_n']=synteny_condensed['hg_seq'].str.count('-')
# synteny_condensed['mm_dash_n']=synteny_condensed['mm_seq'].str.count('-')

# synteny_condensed['hg_percent']=1-(synteny_condensed['hg_dash_n']/synteny_condensed['total_align_len'])
# synteny_condensed['mm_percent']=1-(synteny_condensed['mm_dash_n']/synteny_condensed['total_align_len'])


# print(((synteny_condensed['hg_len'] + synteny_condensed['hg_dash_n']) == synteny_condensed['total_align_len']).all())
# print(((synteny_condensed['mm_len'] + synteny_condensed['mm_dash_n']) == synteny_condensed['total_align_len']).all())

True
True


In [5]:
np.median(synteny_condensed['hg_percent'])

0.9612903225806452

In [6]:
chrom_len=pd.read_csv('/pollard/home/shzhang/akita_variant_scoring/data/chrom_lengths_hg38', sep='\t')

In [7]:
data_path='/pollard/data/projects/kgjoni/CBTN_collab/CBTN_data/'
scores=pd.read_csv(data_path + 'simple-variants_230620/simple_variants_scored/simple_variants_all_scores', sep='\t')
scores['oneminus_corrmean']=1-scores['corr_mean']
scores

,var_index,mse_mean,corr_mean,oneminus_corrmean
0,0,NaN,NaN,NaN
1,1,5.914621e-03,0.979139,2.086145e-02
2,2,1.629834e-07,1.000000,4.470926e-07
3,3,1.445605e-08,1.000000,3.242280e-08
4,4,2.073410e-08,0.999999,9.900652e-07
...,...,...,...,...
214039,214039,4.707555e-08,1.000000,9.518438e-08
214040,214040,NaN,NaN,NaN
214041,214041,NaN,NaN,NaN
214042,214042,NaN,NaN,NaN


In [9]:
#get mse and corr idx 
mse_sort=scores.sort_values(by='mse_mean', ascending=False)
top_1000_mse = mse_sort.index[:500]
top_1000_mse_idx = mse_sort['var_index'].loc[top_1000_mse].tolist()

corr_sort=scores.sort_values(by='oneminus_corrmean', ascending=False)
top_1000_corr = corr_sort.index[:500]
top_1000_corr_idx = corr_sort['var_index'].loc[top_1000_corr].tolist()


In [10]:
idx_union= list(set(top_1000_mse_idx).union(top_1000_corr_idx))
len(idx_union)

988

In [11]:
simple_df=pd.read_csv(data_path + 'simple-variants_230620/simple_variants_all', sep='\t')

In [16]:
#keep only those in top mse and corr
#simple_top = simple_df[simple_df['sample_var_index'].isin(idx_union)]
simple_top=simple_df[simple_df.index.isin(idx_union)]
simple_top

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,BiospecimenID,sample_var_index
133,chr5,20909748,.,TCTC,T,.,PASS,"CAL=Strelka2,VarDict;CSQ=-|intron_variant&non_...",GT:AD:ADC:ADR:AF:AFC:AFR:DP:DPC:DPR:GTC:GT_STATUS,BS_0YWFFRYV,133
1009,chr1,28546633,.,CACGGTGGCTCACGCCTGTAATCCCAGCATTTTGGGAAGCCGAGGT...,C,.,PASS,"CAL=VarDict,Lancet;CSQ=-|downstream_gene_varia...",GT:AD:ADC:ADR:AF:AFC:AFR:DP:DPC:DPR:GTC:GT_STATUS,BS_19EJ85F8,36
1103,chr1,155342274,.,TCA,T,.,PASS,"CAL=Strelka2,Mutect2,VarDict,Lancet;CSQ=-|intr...",GT:AD:ADC:ADR:AF:AFC:AFR:DP:DPC:DPR:GTC:GT_STATUS,BS_19EJ85F8,130
1147,chr1,213640280,.,TTC,T,.,PASS,"CAL=Strelka2,VarDict;CSQ=-|intron_variant&non_...",GT:AD:ADC:ADR:AF:AFC:AFR:DP:DPC:DPR:GTC:GT_STATUS,BS_19EJ85F8,174
1739,chr5,105062135,.,CGT,C,.,PASS,"CAL=Strelka2,VarDict;CSQ=-|intron_variant&non_...",GT:AD:ADC:ADR:AF:AFC:AFR:DP:DPC:DPR:GTC:GT_STATUS,BS_19EJ85F8,766
...,...,...,...,...,...,...,...,...,...,...,...
212617,chrX,121433158,.,ATTACTATCT,A,.,PASS,"CAL=Strelka2,Mutect2,VarDict;CSQ=-|intron_vari...",GT:AD:ADC:ADR:AF:AFC:AFR:DP:DPC:DPR:GTC:GT_STATUS,BS_QR84QJN4,5143
212786,chr1,187435115,.,T,TTA,.,PASS,"CAL=Strelka2,Mutect2,VarDict,Lancet;CSQ=TA|int...",GT:AD:ADC:ADR:AF:AFC:AFR:DP:DPC:DPR:GTC:GT_STATUS,BS_A86314EJ,93
212829,chr2,77613461,.,T,TA,.,PASS,"CAL=Strelka2,Mutect2,VarDict,Lancet;CSQ=A|inte...",GT:AD:ADC:ADR:AF:AFC:AFR:DP:DPC:DPR:GTC:GT_STATUS,BS_A86314EJ,136
212914,chr3,6395149,.,G,GCGCCCCCCCC,.,PASS,"CAL=Mutect2,VarDict;CSQ=CGCCCCCCCC|intergenic_...",GT:AD:ADC:ADR:AF:AFC:AFR:DP:DPC:DPR:GTC:GT_STATUS,BS_A86314EJ,221


In [17]:
#merge with chr len information
chrom_len=pd.read_csv('/pollard/home/shzhang/akita_variant_scoring/data/chrom_lengths_hg38', sep='\t', header=None)
chrom_len[0]='chr' + chrom_len[0]
simple=simple_top.merge(chrom_len, right_on=0, left_on='CHROM', how='left')
simple = simple.reset_index(drop=True)

simple

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,BiospecimenID,sample_var_index,0,1
0,chr5,20909748,.,TCTC,T,.,PASS,"CAL=Strelka2,VarDict;CSQ=-|intron_variant&non_...",GT:AD:ADC:ADR:AF:AFC:AFR:DP:DPC:DPR:GTC:GT_STATUS,BS_0YWFFRYV,133,chr5,181538259
1,chr1,28546633,.,CACGGTGGCTCACGCCTGTAATCCCAGCATTTTGGGAAGCCGAGGT...,C,.,PASS,"CAL=VarDict,Lancet;CSQ=-|downstream_gene_varia...",GT:AD:ADC:ADR:AF:AFC:AFR:DP:DPC:DPR:GTC:GT_STATUS,BS_19EJ85F8,36,chr1,248956422
2,chr1,155342274,.,TCA,T,.,PASS,"CAL=Strelka2,Mutect2,VarDict,Lancet;CSQ=-|intr...",GT:AD:ADC:ADR:AF:AFC:AFR:DP:DPC:DPR:GTC:GT_STATUS,BS_19EJ85F8,130,chr1,248956422
3,chr1,213640280,.,TTC,T,.,PASS,"CAL=Strelka2,VarDict;CSQ=-|intron_variant&non_...",GT:AD:ADC:ADR:AF:AFC:AFR:DP:DPC:DPR:GTC:GT_STATUS,BS_19EJ85F8,174,chr1,248956422
4,chr5,105062135,.,CGT,C,.,PASS,"CAL=Strelka2,VarDict;CSQ=-|intron_variant&non_...",GT:AD:ADC:ADR:AF:AFC:AFR:DP:DPC:DPR:GTC:GT_STATUS,BS_19EJ85F8,766,chr5,181538259
...,...,...,...,...,...,...,...,...,...,...,...,...,...
983,chrX,121433158,.,ATTACTATCT,A,.,PASS,"CAL=Strelka2,Mutect2,VarDict;CSQ=-|intron_vari...",GT:AD:ADC:ADR:AF:AFC:AFR:DP:DPC:DPR:GTC:GT_STATUS,BS_QR84QJN4,5143,chrX,156040895
984,chr1,187435115,.,T,TTA,.,PASS,"CAL=Strelka2,Mutect2,VarDict,Lancet;CSQ=TA|int...",GT:AD:ADC:ADR:AF:AFC:AFR:DP:DPC:DPR:GTC:GT_STATUS,BS_A86314EJ,93,chr1,248956422
985,chr2,77613461,.,T,TA,.,PASS,"CAL=Strelka2,Mutect2,VarDict,Lancet;CSQ=A|inte...",GT:AD:ADC:ADR:AF:AFC:AFR:DP:DPC:DPR:GTC:GT_STATUS,BS_A86314EJ,136,chr2,242193529
986,chr3,6395149,.,G,GCGCCCCCCCC,.,PASS,"CAL=Mutect2,VarDict;CSQ=CGCCCCCCCC|intergenic_...",GT:AD:ADC:ADR:AF:AFC:AFR:DP:DPC:DPR:GTC:GT_STATUS,BS_A86314EJ,221,chr3,198295559


In [18]:
simple = simple.rename(columns={1: 'chr_len'})


simple['POS_300KB']=np.floor(np.maximum(0, simple['POS']-150e3))
simple['END_300KB']=np.floor(np.minimum(simple['chr_len'], simple['POS']+150e3))
simple['POS_1MB']=np.floor(np.maximum(0, simple['POS']-5e5))
simple['END_1MB']=np.floor(np.minimum(simple['chr_len'], simple['POS']+ 5e5))


simple['postoend_1mb']=simple['END_1MB']-simple['POS_1MB']
simple['postoend_300kb']=simple['END_300KB']-simple['POS_300KB']



In [19]:
simple


,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,BiospecimenID,sample_var_index,0,chr_len,POS_300KB,END_300KB,POS_1MB,END_1MB,postoend_1mb,postoend_300kb
0,chr5,20909748,.,TCTC,T,.,PASS,"CAL=Strelka2,VarDict;CSQ=-|intron_variant&non_...",GT:AD:ADC:ADR:AF:AFC:AFR:DP:DPC:DPR:GTC:GT_STATUS,BS_0YWFFRYV,133,chr5,181538259,20759748.0,21059748.0,20409748.0,21409748.0,1000000.0,300000.0
1,chr1,28546633,.,CACGGTGGCTCACGCCTGTAATCCCAGCATTTTGGGAAGCCGAGGT...,C,.,PASS,"CAL=VarDict,Lancet;CSQ=-|downstream_gene_varia...",GT:AD:ADC:ADR:AF:AFC:AFR:DP:DPC:DPR:GTC:GT_STATUS,BS_19EJ85F8,36,chr1,248956422,28396633.0,28696633.0,28046633.0,29046633.0,1000000.0,300000.0
2,chr1,155342274,.,TCA,T,.,PASS,"CAL=Strelka2,Mutect2,VarDict,Lancet;CSQ=-|intr...",GT:AD:ADC:ADR:AF:AFC:AFR:DP:DPC:DPR:GTC:GT_STATUS,BS_19EJ85F8,130,chr1,248956422,155192274.0,155492274.0,154842274.0,155842274.0,1000000.0,300000.0
3,chr1,213640280,.,TTC,T,.,PASS,"CAL=Strelka2,VarDict;CSQ=-|intron_variant&non_...",GT:AD:ADC:ADR:AF:AFC:AFR:DP:DPC:DPR:GTC:GT_STATUS,BS_19EJ85F8,174,chr1,248956422,213490280.0,213790280.0,213140280.0,214140280.0,1000000.0,300000.0
4,chr5,105062135,.,CGT,C,.,PASS,"CAL=Strelka2,VarDict;CSQ=-|intron_variant&non_...",GT:AD:ADC:ADR:AF:AFC:AFR:DP:DPC:DPR:GTC:GT_STATUS,BS_19EJ85F8,766,chr5,181538259,104912135.0,105212135.0,104562135.0,105562135.0,1000000.0,300000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
983,chrX,121433158,.,ATTACTATCT,A,.,PASS,"CAL=Strelka2,Mutect2,VarDict;CSQ=-|intron_vari...",GT:AD:ADC:ADR:AF:AFC:AFR:DP:DPC:DPR:GTC:GT_STATUS,BS_QR84QJN4,5143,chrX,156040895,121283158.0,121583158.0,120933158.0,121933158.0,1000000.0,300000.0
984,chr1,187435115,.,T,TTA,.,PASS,"CAL=Strelka2,Mutect2,VarDict,Lancet;CSQ=TA|int...",GT:AD:ADC:ADR:AF:AFC:AFR:DP:DPC:DPR:GTC:GT_STATUS,BS_A86314EJ,93,chr1,248956422,187285115.0,187585115.0,186935115.0,187935115.0,1000000.0,300000.0
985,chr2,77613461,.,T,TA,.,PASS,"CAL=Strelka2,Mutect2,VarDict,Lancet;CSQ=A|inte...",GT:AD:ADC:ADR:AF:AFC:AFR:DP:DPC:DPR:GTC:GT_STATUS,BS_A86314EJ,136,chr2,242193529,77463461.0,77763461.0,77113461.0,78113461.0,1000000.0,300000.0
986,chr3,6395149,.,G,GCGCCCCCCCC,.,PASS,"CAL=Mutect2,VarDict;CSQ=CGCCCCCCCC|intergenic_...",GT:AD:ADC:ADR:AF:AFC:AFR:DP:DPC:DPR:GTC:GT_STATUS,BS_A86314EJ,221,chr3,198295559,6245149.0,6545149.0,5895149.0,6895149.0,1000000.0,300000.0


In [20]:
variants_300kb=simple
variants_300kb=variants_300kb[['CHROM', 'POS_300KB', 'END_300KB', 'ID']]
variants_300kb['POS_300KB']=variants_300kb['POS_300KB'].astype(int)
variants_300kb['END_300KB']=variants_300kb['END_300KB'].astype(int)
print(len(variants_300kb))
variants_300kb.head()

988


,CHROM,POS_300KB,END_300KB,ID
0,chr5,20759748,21059748,.
1,chr1,28396633,28696633,.
2,chr1,155192274,155492274,.
3,chr1,213490280,213790280,.
4,chr5,104912135,105212135,.


In [21]:
variants_1mb=simple
variants_1mb=variants_1mb[['CHROM', 'POS_1MB', 'END_1MB', 'ID']]
variants_1mb['POS_1MB']=variants_1mb['POS_1MB'].astype(int)
variants_1mb['END_1MB']=variants_1mb['END_1MB'].astype(int)
print(len(variants_1mb))
variants_1mb.head()

988


,CHROM,POS_1MB,END_1MB,ID
0,chr5,20409748,21409748,.
1,chr1,28046633,29046633,.
2,chr1,154842274,155842274,.
3,chr1,213140280,214140280,.
4,chr5,104562135,105562135,.


In [22]:
variant_300kb_BED = BedTool.from_dataframe(variants_300kb)
variant_1mb_BED = BedTool.from_dataframe(variants_1mb)

In [23]:
synteny_BED = BedTool.from_dataframe(synteny_condensed)


In [24]:
#reports only SVs where there is overlap with  gene
#-F ensures that at least 0.5 of gene has overlap with SV 
#use loj in this case, because many SVs overlap multiple syntenic regions  
variant_300kb_synt=variant_300kb_BED.intersect(synteny_BED, loj=True, wo=True).to_dataframe(header=None)
variant_1mb_synt=variant_1mb_BED.intersect(synteny_BED, loj=True, wo=True).to_dataframe(header=None)


In [25]:
variant_300kb_synt.columns = list(variants_300kb.columns) + list(synteny_condensed.columns)
variant_300kb_synt = variant_300kb_synt[variant_300kb_synt['hg_chr'] != '.']
print(len(variant_300kb_synt))
variant_300kb_synt.head()

113678


,CHROM,POS_300KB,END_300KB,ID,hg_chr,hg_start,hg_end,hg_len,hg_seq,mm_chr,mm_start,mm_end,mm_len,mm_seq,total_align_len,hg_dash_n,mm_dash_n,hg_percent,mm_percent
0,chr5,20759748,21059748,.,chr5,20766763,20767471,709,TCTGTAAGCACAATACTTTGTAT---TTTTTGACCAACCTACCTTA...,chr15,81698495,81699204,710,TCTGTAC-CACAATAATCTTTATACCTTTTCGAGCTACCAACTTTC...,744,35,34,0.9529569892473119,0.9543010752688172
1,chr5,20759748,21059748,.,chr5,20767971,20769891,1921,CTGGTTGGTAGATTGCTACCTCCTtgattaacagtggtaaatattt...,chr15,81699205,81701043,1839,CTGGTTACTGGAGTGCTGCCTTCTTGATTAACAGAGTTGAATATTT...,1973,52,134,0.9736441966548404,0.9320831221490117
2,chr5,20759748,21059748,.,chr5,20769910,20769953,44,GATATAACACTTTCATACAAATGTCATGAAATAGCAGACATTTT,chr15,81701273,81701316,44,GGTAAGATATGCTTATATAAATGTCATGTAATAACAAATGTTTT,44,0,0,1.0,1.0
3,chr5,20759748,21059748,.,chr5,20778823,20779823,1001,AATAGACTTGGGAAATGCGATGTTTTCTAGGCATGATCCTGAAATC...,chr15,81708469,81709456,988,AAGAGAGTTAGGAAATATGGT-CTTTCTTGGAGTAAAGCTGAATTA...,1061,60,73,0.943449575871819,0.9311969839773798
4,chr5,20759748,21059748,.,chr5,20780510,20781236,727,aactgtagctgactgATCTACAGAAAGGTTTCATTCACTC-AGCGT...,chr15,81710628,81711251,624,AACTCCACCAGGCTGAC--ATAATCAGGTTTACTTTATTCTGGCAT...,748,21,124,0.9719251336898396,0.8342245989304813


In [26]:
variant_1mb_synt.columns = list(variants_1mb.columns) + list(synteny_condensed.columns)
variant_1mb_synt = variant_1mb_synt[variant_1mb_synt['hg_chr'] != '.']
print(len(variant_1mb_synt))
variant_1mb_synt.head()

372089


,CHROM,POS_1MB,END_1MB,ID,hg_chr,hg_start,hg_end,hg_len,hg_seq,mm_chr,mm_start,mm_end,mm_len,mm_seq,total_align_len,hg_dash_n,mm_dash_n,hg_percent,mm_percent
0,chr5,20409748,21409748,.,chr5,20427363,20427405,43,GAATAAACATCACATATTATCTTAAAAGGCTTATTTTACTCAA,chr15,81368472,81368514,43,GAGTAAACACTGAACATAACCTTAAAAGATTTATTTGGCCCAA,43,0,0,1.0,1.0
1,chr5,20409748,21409748,.,chr5,20428715,20429577,863,atgtttgtGACCCAATGTGTTTTACAATAAGACGTATATAAGCAAA...,chr15,81369922,81370730,809,ATATTTTTTATGCACTATGCTTTT--ACAGAATGCATATATGCATA...,875,12,66,0.9862857142857143,0.9245714285714286
2,chr5,20409748,21409748,.,chr5,20430941,20431130,190,AAAGAATGACAATACATTCCTGCATCTATTTCTCCTAATATATTTA...,chr15,81374457,81374661,205,AAAGAGT---AAGACATTTCT--ATTTAT--CTGTTAATATGTTTG...,219,29,14,0.8675799086757991,0.9360730593607306
3,chr5,20409748,21409748,.,chr5,20431528,20432741,1214,AAAGAAAAgaacctcagtcaatcacaaagagccaaccagcctattg...,chr15,81374698,81375849,1152,AAAACAATGAACCTTGTCAATTTTTGAACAGTAAACCAAT-TGTCA...,1290,76,138,0.9410852713178295,0.8930232558139535
4,chr5,20409748,21409748,.,chr5,20434929,20435479,551,CTTGGAAGGGGTATGCATACATTCAGATGGATAGCCACTGCTTTAT...,chr15,81376313,81376851,539,CTTCAGGAGGAAATGCATTCTGTAAGATGTATTAGCATTGGCCTGC...,574,23,35,0.9599303135888502,0.9390243902439024


In [42]:
def find_index(hg_seq, idx):
    count_letters = 0  # To count the letters encountered in the combined string
    idx_new = 0  # Corresponding index in combined string
    
    for char in hg_seq:
        if count_letters == (idx+1):
            break
        if char.isalpha():
            count_letters += 1
        idx_new += 1
    
    return idx_new

#given two aligned strings with start and end idx
#count up amount of same characters within the alignment subset 
def count_str(hg_seq, mm_seq, start_idx, end_idx):
    #print('start_idx', start_idx)
    #print('end_idx', end_idx)
    hg_subset = hg_seq[start_idx:end_idx]
    mm_subset = mm_seq[start_idx:end_idx]

    count_same = sum(1 for a, b in zip(hg_subset, mm_subset) if a == b)
    
    if end_idx>start_idx:
        prop=count_same/(end_idx-start_idx)
    else:
        prop=0
    
    
    return(prop)

In [ ]:
start=time.time()
for i in simple.index:
   
    if i % 10 == 0:
        print('index ', i)
    
    
    #THIS ONE FOR 1MB SCORE 
    try:
        #print(i)
        variant=simple.loc[i]
        ID=variant.ID
        int_df=variant_1mb_synt[variant_1mb_synt['ID'] == ID]

        #print(len(int_df))
    
    
        if len(int_df)>0:


            int_df['start_idx']=(np.maximum(int_df['POS_1MB'], int_df['hg_start'])) - int_df['hg_start'] 
            int_df['end_idx']=(np.minimum(int_df['END_1MB'], int_df['hg_end'])) - int_df['hg_start'] 
            int_df['n_overlap']=int_df['end_idx']-int_df['start_idx']

            
            #get alignment indices
            int_df['start_align_idx'] = int_df.apply(lambda row: find_index(row['hg_seq'], row['start_idx']), axis=1)
            int_df['end_align_idx'] = int_df.apply(lambda row: find_index(row['hg_seq'], row['end_idx']), axis=1)
            int_df['n_align_overlap']=int_df['end_align_idx']-int_df['start_align_idx']
        
        
            int_df['count_prop'] = int_df.apply(lambda row: count_str(row['hg_seq'], row['mm_seq'], row['start_align_idx'], row['end_align_idx']), axis=1)


            #proportion of SV that covers a syntenic region  
            simple.loc[i, 'synt_prop_overlap_1MB'] = int(np.sum(int_df['n_overlap'])) /abs(simple.loc[i, 'postoend_1mb'])
            #weighted proportion of SV in syntenic region, and how similar those regions are 
            simple.loc[i, 'align_synt_prop_1MB'] =  (np.sum(int_df['count_prop'] *int_df['n_overlap'])) /abs(simple.loc[i, 'postoend_1mb'])



        else:
            simple.loc[i, 'synt_prop_overlap_1MB'] = 0
            simple.loc[i, 'align_synt_prop_1MB'] = 0
            
    except:
        print('error in index', i)
        
        
    #THIS ONE FOR NORMAL SV LENGTH 
    try:
        #print(i)
        variant=simple.loc[i]
        ID=variant.ID
        int_df=variant_300kb_synt[variant_300kb_synt['ID'] == ID]
        #print(len(int_df))
    
    
        if len(int_df)>0:


            int_df['start_idx']=(np.maximum(int_df['POS_300KB'], int_df['hg_start'])) - int_df['hg_start'] 
            int_df['end_idx']=(np.minimum(int_df['END_300KB'], int_df['hg_end'])) - int_df['hg_start'] 
            int_df['n_overlap']=int_df['end_idx']-int_df['start_idx']

            
            #get alignment indices
            int_df['start_align_idx'] = int_df.apply(lambda row: find_index(row['hg_seq'], row['start_idx']), axis=1)
            int_df['end_align_idx'] = int_df.apply(lambda row: find_index(row['hg_seq'], row['end_idx']), axis=1)
            int_df['n_align_overlap']=int_df['end_align_idx']-int_df['start_align_idx']
        
        
            int_df['count_prop'] = int_df.apply(lambda row: count_str(row['hg_seq'], row['mm_seq'], row['start_align_idx'], row['end_align_idx']), axis=1)


            #proportion of SV that covers a syntenic region  
            simple.loc[i, 'synt_prop_overlap_300KB'] = int(np.sum(int_df['n_overlap'])) /abs(simple.loc[i, 'postoend_300kb'])
            #weighted proportion of SV in syntenic region, and how similar those regions are 
            simple.loc[i, 'align_synt_prop_300KB'] =  (np.sum(int_df['count_prop'] *int_df['n_overlap'])) /abs(simple.loc[i, 'postoend_300kb'])



        else:
            simple.loc[i, 'synt_prop_overlap_300KB'] = 0
            simple.loc[i, 'align_synt_prop_300KB'] = 0
            
    except:
        print('error 1mb in index', i)
        
        
end=time.time()
print(f"Time taken: {end-start} seconds")
    

index  0
index  10
index  20
index  30
index  40
index  50
index  60
index  70
index  80
index  90
index  100
index  110
index  120
index  130
index  140
index  150
index  160
index  170
index  180
index  190
index  200
index  210
index  220
index  230
index  250
index  260
index  270
index  280
index  290
index  300
index  310
index  320
index  330
index  340
index  350
index  360
index  370
index  380
index  390
index  400
index  410
index  420
index  430
index  440
index  450
index  460
index  470
index  480
index  490
index  500
index  510
index  520
index  530
index  540
index  550
index  560
index  570
index  580
index  590
index  600
index  610
index  620
index  630
index  640
index  650
index  660
index  670
index  680
index  690
index  700
error in index 706


In [ ]:
simple.to_csv('/pollard/home/shzhang/20231023_simple_top500_synteny_1mb.txt', sep='\t', index=False)  

In [ ]:
simple